# Chat network from messages

After 30 million messages, crawler stopped and we lost some data. Let's retrieve data and create new collection called processed chats and let's change structure of the documents in the network collection. 

Here we are going to parse every message to find mentions of other chats. Each chat will have its own list of mentioned chats. 

Telegram crawler has similar workflow while fetching the messages, thus it creates and save network on the go. Unfortunately, due to the size of the network dictionary that was created in the third iteration after 30 million messages, it crashed as MongoDB cannot write more than 16 MB in one document. Now we are going to retrieve the lost network dataset by finding mentions once again. This is crucial for the crawler to continue its work as these mentions are an input for the next chats that will be fetched. 

This issue was, of course, resolved in the main code, but this notebook is kept as an illustration of what can be done with the Telegram dataset. 

For those who did not check the documentation, each iteration represents one set of chats that are processed separately: 
    - First iteration are the input chats i.e. the chats provided by the user related to, most likely, one topic
    - Second iteration are the chats that are mentioned in the chats procesed in the first iteration, i.e. input chats
    - Third iteration are the chats that are mentioned in the input chats of the second iteration and so on.

Mentioned chats are found either as endpoints (linked chat or forwards) or in the text of the message (strings with @ or links that contain t.me)

In the mean time, structure of network collection has changed and the whole script that interacts with it, thus not only we are going to retrieve network, but we are going to save it differently, so it works with our updated script logic. Our updated script logic also needs another collection to track processed chats and we are going to make it at the end of this notebook.

Finally, this notebook can be used to show how **not** to interact with MongoDB as I did not take into account good practices when it comes to querying MongoDB. You can see that one script took us 12000 seconds to finish!! 

Good practices, good examples and explanation of workflow that speeds up the the data retrieval process is in the notebook called "mongodb-query-tips".

# How not to interact with MongoDB

In [5]:
# Let's get all the chats that were processed in the previous iterations
# We are going to use them to filter messages that does not belong to the third iteration - iteration which was interuppted and which network was lost

from pymongo import MongoClient
import re
import time

# Connect to MongoDB; port, path and namings are the same as the ones defined in the config-database.ini before the crawl was initiated
# Check another notebook with general tips for more
client = MongoClient("mongodb://localhost:27017/")
db = client["Telegram-090124-crawler"]
network_collection = db["network"]

processed_chat_ids = []
cursor = network_collection.find({})

for document in cursor:
    # Iterate over iterations in the document
    for iteration_key, iteration_value in document.items():
        if isinstance(iteration_value, list):
            # Iterate over chat IDs within the iteration
            for chat_id_dict in iteration_value:
                # Extract CHATID values where mentioned
                chat_id = next(iter(chat_id_dict), None)
                if chat_id:
                    processed_chat_ids.append(chat_id)

print(sorted(processed_chat_ids))

['1086654357', '1116064717', '1123235006', '1173589786', '1173589786', '1192181186', '1194730261', '1196259684', '1197014308', '1197014308', '1226804730', '1247885695', '1247885695', '1284383291', '1284383291', '1284383291', '1296890338', '1302523602', '1303089864', '1304292577', '1310294120', '1310294120', '1315767641', '1318919825', '1327284437', '1334204098', '1361114342', '1388884232', '1388884232', '1408165010', '1416713361', '1432763973', '1432763973', '1449950140', '1474224071', '1486129307', '1487078912', '1511032698', '1517200518', '1534626323', '1534626323', '1567891251', '1569004724', '1585919701', '1585919701', '1587754222', '1601237033', '1610648128', '1616881342', '1619444408', '1622229867', '1629235900', '1633917165', '1635259727', '1635259727', '1635724402', '1635724402', '1640554183', '1640554183', '1648865616', '1660680716', '1671964902', '1675990597', '1676667633', '1697010772', '1697010772', '1700136522', '1708243092', '1708243092', '1718632902', '1735420988', '1735

Let's see first few messages to understand better our messages collection.

In [6]:
# Show few messages
messages_collection = db["messages"]

cursor = messages_collection.find({})

# Lets print first few messages
for index, message in enumerate(cursor):
    if index < 2:
        print(message)
    else:
        break

cursor.rewind()

{'_id': ObjectId('659db99fcb4aa618cff2a132'), '_': 'Message', 'id': 16, 'peer_id': {'_': 'PeerChannel', 'channel_id': 1318919825}, 'date': datetime.datetime(2023, 12, 15, 15, 11, 30), 'message': '//t.me/thelastbattle20', 'out': True, 'mentioned': False, 'media_unread': False, 'silent': False, 'post': False, 'from_scheduled': False, 'legacy': False, 'edit_hide': False, 'pinned': False, 'noforwards': False, 'from_id': {'_': 'PeerUser', 'user_id': 5963929426}, 'fwd_from': None, 'via_bot_id': None, 'reply_to': None, 'media': {'_': 'MessageMediaWebPage', 'webpage': {'_': 'WebPage', 'id': 9170793121369551478, 'url': 'https://t.me/thelastbattle20', 'display_url': 't.me/thelastbattle20', 'hash': 0, 'type': 'telegram_channel', 'site_name': 'Telegram', 'title': 'EUROPA - THE LAST BATTLE V2 (plus bonus content)', 'description': '"Communism was not created by the masses to overthrow the bankers, Communism was created by the bankers to overthrow and enslave the masses."', 'photo': {'_': 'Photo', 'i

In [8]:
# Functions for parsing message
def get_fwd_from(message):
    fwd_from_chats = []

    fwd_from = message.get("fwd_from")
    if fwd_from and isinstance(fwd_from, dict):
        from_id = fwd_from.get("from_id")

        if isinstance(from_id, dict):
            if from_id.get("_") == "PeerChannel":
                id_value = from_id.get("channel_id")
                fwd_from_chats.append(id_value)

        elif isinstance(from_id, dict) and from_id.get("_") == "PeerChat":
            id_value = from_id.get("chat_id")
            fwd_from_chats.append(id_value)

    return fwd_from_chats


def get_mentioned_chats(mentioned_chats, message):
    message_text = message.get("message", "")

    if message_text:
        mention_tdotme = re.findall(
            r"(?:(?:https?://)?t\.me/|t\.me/)(\w+)", message_text
        )
        mentioned_chats["mentions_with_tdotme"].extend(mention_tdotme)

        mention_at = re.findall(r"@(\w+)", message_text)
        mentioned_chats["mentions_with_at"].extend(mention_at)

    return mentioned_chats


def get_linked_chat_id(chat_id):
    chats_collection = db["chats"]

    # Find the document with the given chat_id
    chat_document = chats_collection.find_one({"full_chat.id": chat_id})

    if chat_document:
        return chat_document["full_chat"]["linked_chat_id"]
    else:
        return None  # Return None if the linked_chat_id is not found or document doesn't exist


# Record the start time
start_time = time.time()

full_iteration = {}

# Iterate over documents in the collection
for message in cursor:
    chat_id = message.get("peer_id", {}).get("channel_id")

    if chat_id:
        # Check if the message from the chat has been processed in previous iterations
        if chat_id and str(chat_id) not in processed_chat_ids:
            # Check if the chat_id already exists in the full_iteration dictionary
            if str(chat_id) not in full_iteration:
                # Get linked chat_id
                linked_chat_id = get_linked_chat_id(chat_id)

                # Initialize for each new chat_id
                full_iteration[str(chat_id)] = {
                    "Fwd_from": [],
                    "Linked": linked_chat_id,  # Assuming linked_chat_id is defined somewhere
                    "Mentions": {
                        "mentions_with_tdotme": [],
                        "mentions_with_at": [],
                    },  # Initialize for each new chat_id
                }

            fwd_from_chats = get_fwd_from(message)
            full_iteration[str(chat_id)]["Fwd_from"].extend(fwd_from_chats)

            mentioned_chats = get_mentioned_chats(
                full_iteration[str(chat_id)]["Mentions"], message
            )

end_time = time.time()
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Time taken: {elapsed_time} seconds")

Time taken: 11210.855076789856 seconds


In [11]:
# let's save our data after 11210 seconds that took our script to run!
import pickle
import json


# Save to pickle
with open("data_regained/full_iteration.pkl", "wb") as pickle_file:
    pickle.dump(full_iteration, pickle_file)

# Save to JSON
with open("data_regained/full_iteration.json", "w") as json_file:
    json.dump(full_iteration, json_file)

In [12]:
# Save to pickle
with open("data_regained/full_iteration2.pkl", "wb") as pickle_file:
    pickle.dump(full_iteration, pickle_file)

In [28]:
# Let's check how many mentions we found
all_values_list = []

for chat_id, data in full_iteration.items():
    if not isinstance(data, dict):
        continue

    fwd_from_value = data.get("Fwd_from", [])
    linked_value = data.get("Linked", "")

    mentions_data = data.get("Mentions", {})
    tdotme_value = mentions_data.get("mentions_with_tdotme", [])
    at_value = mentions_data.get("mentions_with_at", [])

    values = [fwd_from_value, linked_value, tdotme_value, at_value]

    for value in values:
        if isinstance(value, (list, str)):
            all_values_list.extend(value)
        elif isinstance(value, dict):
            all_values_list.extend(value.values())

all_values_list = list(set(all_values_list))

print("All Values List:", all_values_list)

All Values List: ['NarfGb', 1108344843, 'ky', 'WeThePatriotic', 'fourwatches', 'MParkBoston25', 1304952855, 'joeycamp2021', 1587544096, 'truckers4freedom', 'johnnypatriot1776', 1302855719, 'stroppyme', 'KairosPresse', 'SHYEPPSHA', 'NewPartyChat', 'Altzerogas', 1250426953, 1625817162, 1263009866, 'Owner_FCB_D3CODE', 'PPLPLEASR1', 'Liberta32943400', 'UncleCornPopFren', 'freeworldorder', 'Hyperion_S', 1512046689, 'hr_tradition', 'RotterdamZoo', 'ONTHEMARGINPOD', 'TonyRobbinsLive', 1242562692, 'SarahsandersOfficial', 1582301320, 'REALDONADOTRUMPO', 'HollywoodShooter', 'AwkwardMama', 'Nationalists_United', 'MarilynMonroeReal', 'TuckerCarIsonheIp', 1831338136, 'dominikstapf', 'ResistingThyLies', 1275068572, 'FreedomOutLoud', 'Physics_Facts_Science', 1510473887, 1780482210, 'HaydenCwwg1wga', 'PVACONLINE', 'elcolombiano', 'ParentRyanSJ', 'britainslostlivingpubs', 1058013363, 1719140532, 'TheGamerWord', 1609040060, 1838678204, 'bobbylawrence_1776', 'SenMikeShirkey', 'George_Monkeyman', 'thehots

In [29]:
print(len(all_values_list))

185404


In [30]:
# Save to pickle
with open("data_regained/all_values_list.pkl", "wb") as pickle_file:
    pickle.dump(all_values_list, pickle_file)

# Save to JSON
with open("data_regained/all_values_list.json", "w") as json_file:
    json.dump(all_values_list, json_file)

In [33]:
total_length = 0

for chat_id, data in full_iteration.items():
    if not isinstance(data, dict):
        # Skip non-dictionary entries
        continue

    fwd_from_length = len(data.get("Fwd_from", []))

    # Check if the value for "Linked" is present
    linked_value = data.get("Linked")
    linked_length = 1 if linked_value is not None else 0

    mentions_data = data.get("Mentions", {})
    tdotme_length = len(mentions_data.get("mentions_with_tdotme", []))
    at_length = len(mentions_data.get("mentions_with_at", []))

    total_length += fwd_from_length + linked_length + tdotme_length + at_length

print("Total Length of Values in the Dictionary:", total_length)

Total Length of Values in the Dictionary: 15553982


In [34]:
test_dict = {
    "chatid1": {
        "mentions_with_tdotme": ["breakawaygermans", "breakawaygermans"],
        "othermentions": [],
    },
    "chatid2": {
        "mentions_with_tdotme": ["breakawaygermans", "breakawaygermans"],
        "othermentions": [],
    },
}

for chat_id in test_dict:
    test_dict[chat_id]["iteration_num"] = 3

# Print the modified dictionary
print(test_dict)

{'chatid1': {'mentions_with_tdotme': ['breakawaygermans', 'breakawaygermans'], 'othermentions': [], 'iteration_num': 3}, 'chatid2': {'mentions_with_tdotme': ['breakawaygermans', 'breakawaygermans'], 'othermentions': [], 'iteration_num': 3}}


# Save our data in a new format

We are going to save our lost iteration data, and old iteration data as :

`one_chat_network = {
    "iteration_num": iteration_num,
    "chat_id": chat_id,
    "mentions_with_tdotme": mentions,
    "mentions_with_tdotme": mentions,
    "fwd_from": fwds,
    "linked": linked chats,
}`

In [12]:
import pickle

# Load the pickled dictionary
with open("data_regained/full_iteration.pkl", "rb") as file:
    full_iteration = pickle.load(file)

# Print the first 5 key-value pairs
subset_items = list(full_iteration.items())[:5]

print("Subset of Dictionary Key-Value Pairs:")
for key, value in subset_items:
    print(f"{key}: {value}")

Subset of Dictionary Key-Value Pairs:
1727299590: {'Fwd_from': [1980483891, 1210754102, 1908953484, 1161885109, 1615525104, 1481985424, 1493732793, 1759655246, 1873680626, 1636876924, 1839478523, 1493732793, 1493732793, 1493732793, 1179793684, 1803993713, 1839478523, 1839478523, 1839478523, 1456816010, 1484471145, 1727299590, 1727299590, 1288641123, 1276210696, 1179793684, 1179793684, 1220868865, 1481985424, 1839913190, 2047629848, 1873680626, 1908953484, 1179793684, 1481985424, 1456816010, 1975624760, 1845747187, 1493732793, 1636876924, 1505034921, 1730903921, 1288641123, 1442047675, 1873680626, 2051845256, 1759655246, 1759655246, 1514795572, 1456816010, 1478567133, 1759655246, 1210754102, 1784964695, 1245507567, 1626066743, 1220868865, 1183207683, 1730903921, 1667074244, 1493732793, 1667074244, 1179793684, 1204671678, 1381514061, 1754914557, 1754914557, 1450520527, 1179793684, 1730420550, 1730420550, 1493732793, 1780015731, 1964995934, 1467517334, 1493732793, 1272868362, 1727299590, 

In [12]:
# Load the pickled dictionary
with open("temp_var/iteration_num.pickle", "rb") as file:
    num = pickle.load(file)
    print(num)

3


In [1]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["Telegram-090124-crawler"]
network_collection = db["network"]

In [14]:
# Iterate over the first 3 chats
for i, (main_chat, chat_data) in enumerate(full_iteration.items()):
    if i >= 3:
        break

    new_dict = {
        "iteration_num": 3,
        "chat_id": main_chat,
        "mentions_with_tdotme": chat_data["Mentions"]["mentions_with_tdotme"],
        "mentions_with_at": chat_data["Mentions"]["mentions_with_at"],
        "fwd_from": chat_data["Fwd_from"],
        "linked": chat_data["Linked"],
    }

    print(f"Iteration {i + 1}:")
    print(new_dict)
    print("-" * 50)

Iteration 1:
{'iteration_num': 3, 'chat_id': '1727299590', 'mentions_with_tdotme': ['ImperiumPressOfficial', 'countercurrents', 'remnantposter', 'countercurrents', 'countercurrents', 'countercurrents', 'countercurrents', 'ImperiumPressOfficial', 'countercurrents', 'remnantposter', 'countercurrents', 'countercurrents', 'countercurrents', 'countercurrents'], 'mentions_with_at': ['countercurrents', 'heywildrich', 'countercurrents', 'DarkAcademia', 'EsotericBowdenism', 'TheAyatollah', 'countercurrents', 'counter', 'EsotericBowdenism', 'countercurrents', 'countercurrents', 'countercurrents', 'countercurrents', 'heywildrich', 'countercurrents', 'DarkAcademia', 'EsotericBowdenism', 'TheAyatollah', 'countercurrents', 'counter', 'EsotericBowdenism', 'countercurrents', 'countercurrents', 'countercurrents'], 'fwd_from': [1980483891, 1210754102, 1908953484, 1161885109, 1615525104, 1481985424, 1493732793, 1759655246, 1873680626, 1636876924, 1839478523, 1493732793, 1493732793, 1493732793, 1179793684

In [15]:
# Insert new data
for main_chat, chat_data in full_iteration.items():
    new_dict = {
        "iteration_num": 3,
        "chat_id": int(main_chat),
        "mentions_with_tdotme": chat_data["Mentions"]["mentions_with_tdotme"],
        "mentions_with_at": chat_data["Mentions"]["mentions_with_at"],
        "fwd_from": chat_data["Fwd_from"],
        "linked": chat_data["Linked"],
    }

    network_collection.insert_one(new_dict)

In [24]:
# Let's get iteration 1 data
document = network_collection.find_one({})
if document and "iteration1" in document:
    iteration1_data = document["iteration1"]
    print(iteration1_data)

[{'1318919825': {'mentions_with_tdotme': ['thelastbattle20', 'thelastbattle20', 'thelastbattle20', 'PutinistRussia', 'putinistrussia', 'PutinistRussia', 'PutinistRussia', 'thelastbattle20'], 'mentions_with_at': ['covid_vaccine_injuries', 'https', 't', 'putinistrussia'], 'Fwd_from': [1976161109, 1976161109], 'Linked': None}}, {'1920077960': {'mentions_with_tdotme': [], 'mentions_with_at': [], 'Fwd_from': [], 'Linked': None}}, {'1123235006': {'mentions_with_tdotme': ['Agartha_dump'], 'mentions_with_at': ['NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'slavicvolunteersww2', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMusic88', 'NsaMus

In [17]:
# Let's write in the database iteration 1 data
for data_dict in iteration1_data:
    chat_id, chat_data = next(iter(data_dict.items()))
    new_dict = {
        "iteration_num": 1,
        "chat_id": int(chat_id),
        "mentions_with_tdotme": chat_data.get("mentions_with_tdotme"),
        "mentions_with_at": chat_data.get("mentions_with_at"),
        "fwd_from": chat_data.get("Fwd_from"),
        "linked": chat_data.get("Linked"),
    }

    network_collection.insert_one(new_dict)

In [6]:
# Let's now extract mentions from iteration 2
documents = network_collection.find({})

for document in documents:
    iteration2_data = document.get("iteration2")

    if iteration2_data and isinstance(iteration2_data, list):
        for obj in iteration2_data:
            chat_id, chat_data = next(iter(obj.items()))

            new_dict = {
                "iteration_num": 2,
                "chat_id": int(chat_id),
                "mentions_with_tdotme": chat_data.get("mentions_with_tdotme"),
                "mentions_with_at": chat_data.get("mentions_with_at"),
                "fwd_from": chat_data.get("Fwd_from"),
                "linked": chat_data.get("Linked"),
            }

            network_collection.insert_one(new_dict)

In [24]:
# Save old iteration data
old_network_collection = db["old_network_collection"]

# Query to filter old network
query = {"$or": [{"iteration1": {"$exists": True}}, {"iteration2": {"$exists": True}}]}
# Project to reshape the documents
projection = {"_id": 1, "iteration1": 1, "iteration2": 1}

# Transfer documents using the aggregate method
pipeline = [{"$match": query}, {"$out": "old_network_collection"}]
network_collection.aggregate(pipeline)

In [25]:
# Let's delete our old documents from the network
query_condition = {
    "$or": [
        {"iteration1": {"$exists": True}},
        {"iteration2": {"$exists": True}},
    ]
}

# Delete documents from the filter
result = network_collection.delete_many(query_condition)

# Print the number of documents deleted
print(f"Deleted {result.deleted_count} documents")

Deleted 2 documents


In [33]:
chats_collection = db["chats"]

distinct_chats = chats_collection.distinct("full_chat.id")
print(len(distinct_chats))

2676


In [34]:
distinct_chats = network_collection.distinct("chat_id")
print(len(distinct_chats))

2663


### Processed chats collection 

New script has new logic to check for processed chats, so let's create a new collection with processed chats from our pickle that stores processed chats.

In [5]:
import pickle

with open("temp_var/processed_chats.pickle", "rb") as file:
    processed_chats = pickle.load(file)

In [6]:
for key in processed_chats:
    print(f"{key}:{len(processed_chats[key])}")

valid_processed_chats:2674
invalid_processed_chats:177
valid_processed_entities:3451
invalid_processed_entities:1561


New collection with processed chats has next structure that we are going to recreate:

`        {
            "iteration_num": iteration_num,
            "type": type,
            "username": entity,
            "chat_id": chat_id,
        }`



In [7]:
print(processed_chats["valid_processed_chats"])

{1277476865, 1368449025, 1769209859, 1288445956, 1810350085, 1727299590, 1785348100, 1280786436, 1677221904, 1396523026, 1287839762, 1779589139, 1355849749, 1597063194, 1286234150, 1568112680, 1331593257, 1254678569, 1410736173, 1701929005, 1275396146, 1372397619, 1431969843, 1050820672, 1750368327, 1157742664, 1625817162, 1628651599, 1378394192, 1545306191, 1306353747, 1520762965, 1004953685, 1576091739, 1960452188, 1540538461, 1413054563, 1310294120, 1267531884, 1235779695, 1850392688, 1517002867, 1681539189, 1408548985, 1354301573, 1467605127, 1710801032, 1188544652, 1159266445, 1443119244, 1255694477, 1455227024, 1114095761, 1488339094, 1161789591, 1317863577, 1433698461, 1385914528, 1688608929, 1524146345, 1808670894, 1169129649, 1297965235, 1194025145, 1435795644, 1776197820, 1199628484, 1588658373, 1167712455, 1266622669, 1313218767, 1327284437, 1495662806, 1335927003, 1161666782, 1773879520, 1500446948, 1459126503, 1430102248, 1181417705, 1440129259, 1172275438, 1377378548, 135

In [8]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["Telegram-090124-crawler"]

processed_chats_collection = db[
    "processed_chats"
]  # Create a new collection for processed chats

all_proc = []

for key in processed_chats:
    if key == "invalid_processed_chats":
        for item in processed_chats[key]:
            username = None
            chat_id = None
            if isinstance(item, str):
                username = item
            elif isinstance(item, int):
                chat_id = item
            new_dict = {
                "iteration_num": 3,
                "type": "invalid_chats",
                "username": username,
                "chat_id": chat_id,
            }
            processed_chats_collection.insert_one(new_dict)

    if key == "invalid_processed_entities":
        for item in processed_chats[key]:
            username = None
            chat_id = None
            if isinstance(item, str):
                username = item
            elif isinstance(item, int):
                chat_id = item
            new_dict = {
                "iteration_num": 3,
                "type": "invalid_entities",
                "username": username,
                "chat_id": chat_id,
            }
            processed_chats_collection.insert_one(new_dict)

    if key in ["valid_processed_entities", "valid_processed_chats"]:
        all_proc.extend(list(processed_chats[key]))

all_proc = [word.casefold() if isinstance(word, str) else word for word in all_proc]
all_proc = list(set(all_proc))

for item in all_proc:
    username = None
    chat_id = None
    if isinstance(item, str):
        username = item
    elif isinstance(item, int):
        chat_id = item
    new_dict = {
        "iteration_num": 3,
        "type": "valid",
        "username": username,
        "chat_id": chat_id,
    }
    processed_chats_collection.insert_one(new_dict)